# Prediksi Hilal

In [1]:
# _jengkolrebus
# Juni, 2020
# Curup, Bengkulu

Koordinat kota:
- Yogyakarta: 7.7956 S, 110.3695 E, <a href="https://www.google.com/search?q=koordinat+yogyakarta&rlz=1C1CHBF_enID904ID904&oq=koordinat+yogyakarta&aqs=chrome..69i57.5458j0j7&sourceid=chrome&ie=UTF-8">sumber</a>
- Sabang: 5.892386 N, 95.323706 E, sumber: google maps
- Merauke: 8.499324 S, 140.404902 E, sumber: google maps

In [2]:
from skyfield import almanac
from skyfield.api import load, Topos
from skyfield.units import Angle

from datetime import datetime, timedelta
from pytz import timezone

from ipywidgets import widgets, interactive
from IPython.display import display, HTML

import pandas as pd
import calendar

In [3]:
jkt = timezone('Asia/Jakarta')
ts = load.timescale()
e = load('de421.bsp')


In [4]:
YOG = '7.7956 S','110.3695 E'
SAB = Topos('5.892386 N', '95.323706 E')
MER = Topos('8.499324 S', '140.404902 E')

In [5]:
class Find():
    moon_astrometric = []
    def __init__(self, t0, t1, topo):
        self.t0 = t0
        self.t1 = t1
        self.topo = Topos(topo[0], topo[1])
        self.loc = e['earth'] + self.topo
        self.earth = e['earth']
        self.moon = e['moon']
        self.sun = e['sun']
        self.t_sunset = None
        
    def konjungsi(self):
        t0 = ts.utc(self.t0)
        t1 = ts.utc(self.t1)
        f = almanac.oppositions_conjunctions(e, self.moon)
        t, y = almanac.find_discrete(t0, t1, f)
        for ti, yi in zip(t, y):
            if(yi == 1):
                return ti
            else:
                pass
    def sunset(self, t):
        t = t.utc
        t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
        t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
        f = almanac.sunrise_sunset(e, self.topo)
        t, y = almanac.find_discrete(t0, t1, f)
        for ti, yi in zip(t, y):
            if(yi == False):
                self.t_sunset = ti
                return ti
    
    def pos(self, obj):
        astrometric = self.loc.at(self.t_sunset).observe(e[obj])
        alt, az, d = astrometric.apparent().altaz()
        self.moon_astrometric.append(astrometric)
        return astrometric, alt, az, d.km

In [12]:
def result(t0, t1):
    Find.moon_astrometric.clear()
    a = Find(t0, t1, YOG)
    b = a.konjungsi()
    sunset = a.sunset(b)
    hilal = a.pos('moon')
    print(hilal[1])
    print(a.moon_astrometric)

In [13]:
hari_ini = datetime.now().date()
days_in_month = calendar.monthrange(hari_ini.year, hari_ini.month)[1]
bulan_depan = hari_ini + timedelta(days_in_month)

dari = widgets.DatePicker(value=hari_ini,
    description='Dari:',
    disabled=False
)
sampai = widgets.DatePicker(value=bulan_depan,
    description='Sampai:',
    disabled=False
)

interactive(result, t0=dari, t1=sampai)

interactive(children=(DatePicker(value=datetime.date(2020, 6, 8), description='Dari:'), DatePicker(value=datet…